In [115]:
import pandas as pd
import numpy as np
from binance.client import Client
from datetime import datetime, timedelta, timezone
import os
import pandas_ta as ta
from tqdm import tqdm
import time 
import pytz
import os
# os.system('cls' if os.name == 'nt' else 'clear')
from IPython.display import clear_output
# clear_output(wait=True)
import importlib
import BaseFunctions
importlib.reload(BaseFunctions)
from BaseFunctions import *

import StrategyList
importlib.reload(StrategyList)
from StrategyList import *
# Load API credentials from environment variables
API_KEY = os.getenv("BinanceAPI_250502")
API_SECRET = os.getenv("BinanceSecret_250502")

import warnings
warnings.filterwarnings("ignore")

client = Client(API_KEY, API_SECRET) if API_KEY and API_SECRET else Client()

BackTime = "2025-05-01 00:00:00"
# Convert string to naive datetime object
BackTime = datetime.strptime(BackTime, '%Y-%m-%d %H:%M:%S')
# Make it timezone-aware (UTC)
BackTime = BackTime.replace(tzinfo=timezone.utc)
# print(BackTime)

BackTime = int(BackTime.timestamp() * 1000)  # Convert to milliseconds
Interval = '5m'
current_time = time.time()
pair = 'BTCUSDT'

BuySellFlag = 'Hold'
BuyCounter = 0
SellCounter = 0

In [116]:
BackCandles = get_candles_data(pair, Interval, 1000, BackTime, client)
# Metric = calculate_metrics(BackCandles, pair)
SCORE = allstrategiesv2(BackCandles, pair, client, BackTime)

# print(SCORE)

Error: Required columns missing in DataFrame


KeyError: 'High_4H'